<a href="https://colab.research.google.com/github/mishitamaggo/Automated-Outreach-System/blob/main/Automated_Outreach_System.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Install all required packages
!pip install -q requests beautifulsoup4 gspread oauth2client yagmail google-search-results

print("✓ All packages installed successfully!")

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 385.7/385.7 kB 9.4 MB/s eta 0:00:00
✓ All packages installed successfully!


In [2]:
from google.colab import files
import os

print("📤 Upload your credentials.json file:")
uploaded = files.upload()

# Verify upload
if 'credentials.json' in uploaded or any('credentials' in f for f in uploaded.keys()):
    print("✓ Credentials file uploaded successfully!")
else:
    print("⚠️ Warning: Make sure the file is named 'credentials.json'")


📤 Upload your credentials.json file:


Saving credentials.json to credentials.json
✓ Credentials file uploaded successfully!


In [3]:
# ==================== YOUR CONFIGURATION ====================
# IMPORTANT: Update these values before running!

SENDER_EMAIL = "mishitamaggo23@gmail.com"       # Your Gmail address
SENDER_PASSWORD = "djpshvfjlyorviuu"            # Gmail App Password (no spaces)
SPREADSHEET_NAME = "Outreach Log"               # Your Google Sheet name
SEARCH_QUERY = "UAE e-commerce brands"          # Search query
NUM_RESULTS = 5                                 # Number of brands to find
SERPAPI_KEY = "b2696f69e610d46016664216366411e1034347b6712f0119919e7b351ed12013"

print("✓ Configuration loaded!")
print(f"  Email: {SENDER_EMAIL}")
print(f"  Sheet: {SPREADSHEET_NAME}")
print(f"  Query: {SEARCH_QUERY}")


✓ Configuration loaded!
  Email: mishitamaggo23@gmail.com
  Sheet: Outreach Log
  Query: UAE e-commerce brands


In [5]:
import requests
import re
import time
from datetime import datetime, timedelta
from bs4 import BeautifulSoup
import gspread
from oauth2client.service_account import ServiceAccountCredentials
import yagmail
from serpapi import GoogleSearch

# ==================== STEP 1: FIND BRANDS ====================
def scrape_potential_clients(query, num_results=5):
    """Find potential clients using SerpAPI"""
    print(f"\n{'='*60}")
    print(f"[STEP 1] Searching for: {query}")
    print(f"{'='*60}")

    try:
        params = {
            "q": query,
            "num": num_results,
            "api_key": SERPAPI_KEY,
            "engine": "google"
        }

        search = GoogleSearch(params)
        results_data = search.get_dict()
        organic_results = results_data.get("organic_results", [])

        brands = []
        for result in organic_results:
            url = result.get('link', '')

            # Skip social media and non-business sites
            skip_domains = ['facebook.com', 'instagram.com', 'linkedin.com',
                          'youtube.com', 'twitter.com', 'wikipedia.org',
                          'amazon.com', 'alibaba.com']

            if any(domain in url.lower() for domain in skip_domains):
                continue

            brands.append({
                'name': result.get('title', 'Unknown')[:50],  # Limit name length
                'url': url,
                'emails': [],
                'social_links': {},
                'timestamp': datetime.now().strftime('%Y-%m-%d %H:%M:%S')
            })

        print(f"✓ Found {len(brands)} potential clients")
        return brands

    except Exception as e:
        print(f"✗ Error in search: {e}")
        print("Using fallback test data...")
        return [
            {'name': 'Test Brand 1', 'url': 'https://example.com',
             'emails': [], 'social_links': {},
             'timestamp': datetime.now().strftime('%Y-%m-%d %H:%M:%S')}
        ]

# ==================== STEP 2: EXTRACT EMAILS ====================
def extract_emails(url):
    """Extract email addresses from website"""
    print(f"  [STEP 2] Extracting emails...")
    emails = set()

    try:
        headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36'}
        response = requests.get(url, timeout=10, headers=headers)
        html = response.text
        soup = BeautifulSoup(html, 'html.parser')

        # Find mailto links
        for a in soup.find_all('a', href=lambda x: x and x.startswith('mailto:')):
            email = a['href'][7:].split('?')[0]
            emails.add(email)

        # Regex pattern for emails
        email_pattern = r"[A-Za-z0-9._%+-]+@[A-Za-z0-9.-]+\.[A-Za-z]{2,}"
        found_emails = re.findall(email_pattern, html)
        emails.update(found_emails)

        # Filter out junk
        filtered = [e for e in emails if not any(
            x in e.lower() for x in ['example.com', 'sampleemail', 'test@',
                                      'noreply', 'privacy@', 'support@google']
        )]

        result = list(filtered)[:2]  # Max 2 emails
        print(f"  ✓ Found {len(result)} email(s)")
        return result

    except Exception as e:
        print(f"  ⚠ Could not extract emails: {str(e)[:50]}")
        return []

# ==================== STEP 3: FIND SOCIAL MEDIA ====================
def find_social_links(url):
    """Find social media profiles"""
    print(f"  [STEP 3] Finding social media...")
    socials = {}

    try:
        headers = {'User-Agent': 'Mozilla/5.0'}
        response = requests.get(url, timeout=10, headers=headers)
        soup = BeautifulSoup(response.text, 'html.parser')

        for a in soup.find_all('a', href=True):
            href = a['href'].lower()

            if "instagram.com/" in href and 'instagram' not in socials:
                if not any(x in href for x in ['/p/', '/reel/', '/tv/']):  # Avoid post links
                    socials['instagram'] = a['href']
            elif "facebook.com/" in href and 'facebook' not in socials:
                socials['facebook'] = a['href']
            elif "linkedin.com/company" in href and 'linkedin' not in socials:
                socials['linkedin'] = a['href']

        print(f"  ✓ Found {len(socials)} social profile(s)")
        return socials

    except Exception as e:
        print(f"  ⚠ Could not find social links: {str(e)[:50]}")
        return {}

def personalize_email(brand):
    """Highly personalized email with specific details"""

    # Clean company name
    company_name = brand['name']
    if '|' in company_name:
        company_name = company_name.split('|')[0].strip()
    if '-' in company_name and len(company_name) > 50:
        company_name = company_name.split('-')[0].strip()

    # Get domain name for personalization
    try:
        from urllib.parse import urlparse
        domain = urlparse(brand['url']).netloc.replace('www.', '')
    except:
        domain = brand['url']

    # Analyze their presence
    has_instagram = 'instagram' in brand.get('social_links', {})
    social_count = len(brand.get('social_links', {}))

    # Create highly specific opening
    if has_instagram:
        hook = f"I've been following the e-commerce space in the UAE, and your brand stood out to me while browsing Instagram."
        insight = f"Your products look great, but I noticed your Reels aren't getting the reach they deserve. With a few strategic tweaks to your content format and posting schedule, I believe we could 3-5x your engagement in 30 days."
    else:
        hook = f"I came across your brand, and I'm impressed by what you're building."
        insight = f"Here's the thing—your ideal customers are scrolling Instagram right now, looking for brands exactly like yours. But if you're not showing up in their feed with engaging Reels, they're buying from your competitors instead."

    # Build the email
    template = f"""Hi,

{hook}

{insight}

I run Genixovate, where I help e-commerce brands turn Instagram into their #1 revenue channel through:

✓ Reels that stop the scroll (trending formats + proven hooks)
✓ Content calendars designed for consistency without burnout
✓ DM automation & engagement strategies that convert followers into customers

I'd love to send you a free 5-minute video audit of your current presence with specific recommendations you can implement immediately.

No pressure—just actionable insights you can use whether we work together or not.

Interested?

Best Regards,
Mishita
Genixovate
"""
    return template

# ==================== STEP 5: SEND EMAIL & LOG ====================
def setup_google_sheet():
    """Connect to Google Sheets"""
    try:
        scope = ["https://spreadsheets.google.com/feeds",
                 "https://www.googleapis.com/auth/drive"]
        creds = ServiceAccountCredentials.from_json_keyfile_name(
            "credentials.json", scope
        )
        client = gspread.authorize(creds)
        sheet = client.open(SPREADSHEET_NAME).sheet1

        # Create headers if new sheet
        if sheet.row_count == 0 or sheet.cell(1, 1).value != 'Brand Name':
            sheet.insert_row([
                'Brand Name', 'URL', 'Email', 'Instagram',
                'Status', 'Timestamp', 'Follow Up'
            ], 1)

        print("✓ Connected to Google Sheets")
        return sheet

    except Exception as e:
        print(f"✗ Google Sheets error: {e}")
        return None

def send_email_and_log(brand, recipient_email, sheet):
    """Send email and log to sheet"""
    print(f"  [STEP 5] Sending to {recipient_email}...")

    try:
        # Send email
        yag = yagmail.SMTP(SENDER_EMAIL, SENDER_PASSWORD)
        email_body = personalize_email(brand)

        yag.send(
            to=recipient_email,
            subject="Quick Question About Your Social Media",
            contents=email_body
        )

        # Log to sheet
        if sheet:
            ig_link = brand.get('social_links', {}).get('instagram', 'None')
            sheet.append_row([
                brand['name'],
                brand['url'],
                recipient_email,
                ig_link,
                'Sent',
                brand['timestamp'],
                ''
            ])

        print(f"  ✓ Email sent and logged!")
        return True

    except Exception as e:
        print(f"  ✗ Email error: {str(e)[:100]}")
        if "Application-specific password" in str(e):
            print("  → You need a Gmail App Password! See: https://myaccount.google.com/apppasswords")
        return False

# ==================== MAIN EXECUTION ====================
def main():
    """Run the complete automation"""
    print("\n" + "="*60)
    print("🤖 AUTOMATED CLIENT OUTREACH SYSTEM")
    print("="*60 + "\n")

    # Setup
    sheet = setup_google_sheet()

    # Find brands
    brands = scrape_potential_clients(SEARCH_QUERY, NUM_RESULTS)

    if not brands:
        print("\n❌ No brands found. Check your SerpAPI key or search query.")
        return

    print(f"\n📊 Processing {len(brands)} brand(s)...\n")

    # Process each brand
    success_count = 0
    for idx, brand in enumerate(brands, 1):
        print(f"\n{'─'*60}")
        print(f"📌 Brand {idx}/{len(brands)}: {brand['name']}")
        print(f"{'─'*60}")

        # Extract data
        brand['emails'] = extract_emails(brand['url'])
        brand['social_links'] = find_social_links(brand['url'])

        # Send email if we have an address
        if brand['emails']:
            if send_email_and_log(brand, brand['emails'][0], sheet):
                success_count += 1
            time.sleep(3)  # Rate limiting
        else:
            print(f"  ⚠ No email found for {brand['name']}")

    # Summary
    print(f"\n" + "="*60)
    print(f"✅ COMPLETED!")
    print(f"="*60)
    print(f"📧 Emails sent: {success_count}/{len(brands)}")
    print(f"📊 Check your '{SPREADSHEET_NAME}' sheet for full log")
    print("="*60 + "\n")

# Run the automation
if __name__ == "__main__":
    main()



🤖 AUTOMATED CLIENT OUTREACH SYSTEM

✓ Connected to Google Sheets

[STEP 1] Searching for: UAE e-commerce brands
✓ Found 4 potential clients

📊 Processing 4 brand(s)...


────────────────────────────────────────────────────────────
📌 Brand 1/4: Top 10 eCommerce Companies in Dubai, UAE [2025 Tes
────────────────────────────────────────────────────────────
  [STEP 2] Extracting emails...
  ✓ Found 2 email(s)
  [STEP 3] Finding social media...
  ✓ Found 2 social profile(s)
  [STEP 5] Sending to francis@globalmedia.ae...
  ✓ Email sent and logged!

────────────────────────────────────────────────────────────
📌 Brand 2/4: Best 11 eCommerce Companies in Dubai, UAE
────────────────────────────────────────────────────────────
  [STEP 2] Extracting emails...
  ✓ Found 0 email(s)
  [STEP 3] Finding social media...
  ✓ Found 2 social profile(s)
  ⚠ No email found for Best 11 eCommerce Companies in Dubai, UAE

────────────────────────────────────────────────────────────
📌 Brand 3/4: E-Commerce Com